In [13]:
import numpy as np
import limiters as lim

In [14]:
nx = 100
nghost = 2
nxt = nx+2*nghost
xl, xr = 0., 100.
dx = (xr - xl) / nx
nu = 0.8
c = 1

#'lax-wendroff', 'upwind'
scheme = 'lax-wendroff'

#'c-upwind', 'c-lax-wendroff'
out_dir = 'lw-superbee'

limiter = 'superbee'

dt = nu * dx / c
t = 0.
t_end = 30
nstep = 0
n_output = 1

In [15]:
#Node points
xn = np.linspace(xl-nghost*dx, xr+nghost*dx, nxt+1)

#Cell-centered points
xc = np.linspace(xl-0.5*nghost*dx, xr+0.5*nghost*dx, nxt)

In [16]:
flim = getattr(lim, limiter) if limiter else None

In [17]:
#Initial conditions
u = np.where(xc<50., 1., 0.)

In [18]:
while t < t_end:
    u[:nghost] = u[nghost]
    u[-nghost:] = u[-nghost-1]
    
    u_left = u[nghost-1:-nghost]
    u_right = u[nghost:-nghost+1]
    fc_left = c * u_left
    fc_right = c * u_right
    
    if scheme == 'lax-wendroff':
        if flim:
            u_inner = u[nghost-1:-nghost+1]
            u_left = u[nghost-2:-nghost]
            u_right = u[nghost:-nghost+2] if nghost > 2 else u[nghost:]
            flim_n = flim(u_left, u_inner, u_right)
            flim_l = flim_n[:-2]
            flim_r = flim_n[1:-1]
            
        else:
            flim_r = flim_l = 1.
            
        #low res flux
        fn_low = fc_left
        fn_low_right = fn_low[1:]
        fn_low_left = fn_low[:-1]
            
        #high res flux
        fn_high = 0.5 * (1. - nu) * (fc_right - fc_left)
        fn_high_right = fn_high[1:] * flim_r
        fn_high_left = fn_high[:-1] * flim_l
            
        #left and right fluxes are fn_low + fn_high
        fn_right = fn_low_right + fn_high_right
        fn_left = fn_low_left + fn_high_left
            
        
    else:
        print('Error: Unknown scheme. Exsiting.')
        exit()
        
    u_inner = u[nghost:-nghost]
    u_inner = u_inner - nu * (fn_right - fn_left)
    
    u = np.concatenate((nghost*[0], u_inner, nghost*[0]))

    t = t + dt
    nstep = nstep + 1
    
    if nstep % n_output == 0:
        u_inner.tofile(f'{out_dir}/{scheme}-{int(nstep/n_output):0>4d}.dat')
        